# Python libraries
First of all we start by installing different libraries to be used throughout the project, we use libraries like:
*   pandas and numpy for handling datasets
*   sklearn for the use of TfidfVectorizer and cosine_similarity
*   gensim for the natural language processing
*   networkx for the graph







In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
from nltk.tokenize import sent_tokenize
import re

!pip install --upgrade gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx

# The Data
For this example we will be using aleatory sentences for product comments. Is necessary separate the comments in sentences.

In [17]:
sentences1 = ['Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.',
              'eu consegui comprar esse lindo copo de acrílico.',
              'SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANELAS ELÉTRICAS.  COSTUMO USAR OUTRA PANELA PARA COZIMENTO DE ARROZ (JAPONESA), MAS LEVA MUITO TEMPO,  +/- 50 MINUTOS.  NESSA PANELA  É MUITO MAIS RÁPIDO, EXATAMENTE 6 MINUTOS.    EU RECOMENDO.', 
              'A entrega foi no prazo, as americanas estão de parabéns. A smart tv é muito boa, a navegação na internete e pelos aplicativos e excelente, não trava, sem falar da imagem que é de surpreender. recomendo.',
              'até o momento esse produto não chegou em minha residência', 
              'Produto bom, porém longe de excelente',
              'Comprei a prancha babyliss Pro Nano dia 27/02 é até o momento não recebi o produto o prazo de entrega foi até 16/03. Nunca tive problema com a site americanas mais essa compra foi com o parceiro Miracurl que não cumpriu com seu deveres. Estou ratreando o produto no site do correios e o que informa que o produto não chegou no correio de São Gonçalo. O mínimo que a Miracurl podia fazer é enviar o produto de novo via transportadora ou Sedex 10 para cumprir com o cliente. Só quero que entregue o produto ou então que reembolso o dinheiro.Pois a compra foi paga via cartão. Insatisfeita com parceiro da americana. Produto da empresa Miracurl não compro mais.', 
              'Aparelho atende bem as necessidades do dia-a-dia, sem requerer maiores dificuldades ou esforço para operar.',
              'Comprei pela internet e em nenhum momento falava que o engate da antena não é de antena. ou seja, a gente paga 32,00 pelo produto, mais uns 20 de frete e agora vou ter de gasstar mais 20 num ""ADAPTADOR"" , pois o engate não dá certo na TV ... ENTAO, por que está escrito ANTENA PRA TV DIGITAL se o ENGATE não é de TV??? Se pudesse devolveria e mandaria pro lixo e reaver o dinheiro... Só de pensar na BURROCRACIA, desisto, além do LIXO DE ENTREGA, que demorou quase UM MES, isso dentro da normalidade .... Imagina devolver um produto e tentar REAVER o R$ colocado fora ...', 
              'O produto não gela. Apenas deixa frio. Nunca coloquei nada natural para ver se gela. Sempre tirei as coisas direto da geladeira. Na parte da tarde que é mais quente água por exemplo está apenas fria. Mas achei melhor que garrafa térmica, mas com investimento bem maior também.',
              'Fiquei um pouco decepcionada com o produto. A cadeirinha é bonita e confortavel, como mostrado na foto.   Ela enquadra a crianca em 3 categorias de acordo com o peso. Minha filha tem 4 anos é alta e possui 25 quilos, possui as coxas grossas com isso o cinto da cadeirinha nao a prende mesmo afrouxando o cinto. Consegui inclui la na categoria 2 em que a cadeirinha fica solta no carro e serve como um assento para a criança que é presa pelo cinto de segurança do carro. Como minha filha é alta, ficou certinho mais se fosse baixa estatura não daria certo. Fiquei incomodada com essa adequacao pois nao é descrita no ato da venda e a inclinação prometida nao entra nessa categoria. A criança fica sentada ereta e a cabecinha nao tem tanto apoio.  O manual é descrito de forma complexa, poderia ser mais simples.  A entrega foi no prazo estipulado, nem antes e nem depois.']

In [18]:
sentences = []
for s in sentences1:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]
sentences

['Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.',
 'eu consegui comprar esse lindo copo de acrílico.',
 'SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANELAS ELÉTRICAS.',
 'COSTUMO USAR OUTRA PANELA PARA COZIMENTO DE ARROZ (JAPONESA), MAS LEVA MUITO TEMPO,  +/- 50 MINUTOS.',
 'NESSA PANELA  É MUITO MAIS RÁPIDO, EXATAMENTE 6 MINUTOS.',
 'EU RECOMENDO.',
 'A entrega foi no prazo, as americanas estão de parabéns.',
 'A smart tv é muito boa, a navegação na internete e pelos aplicativos e excelente, não trava, sem falar da imagem que é de surpreender.',
 'recomendo.',
 'até o momento esse produto não chegou em minha residência',
 'Produto bom, porém longe de excelente',
 'Comprei a prancha babyliss Pro Nano dia 27/02 é até o momento não recebi o produto o prazo de entrega foi até 16/03.',
 'Nunca tive problema com a site americanas mais essa compra foi com o parceiro Miracurl que não

# TextRank Algorithm 

Process the text eliminating stop words, minimizing the whole sentence,eliminating special characters

In [19]:
with open('stop_words_portuguese.txt') as infile:
    stopWords = set([line.strip() for line in infile])
    
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("\W", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stopWords])
    return sen_new

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

#clean_sentences

Find a vector representation for each sentence.

In [20]:
#calcular el tf*idf
vectorizer = TfidfVectorizer(stop_words=stopWords)
#X = vectorizer.fit_transform(sentences)

#calcular matrix E
analyzer = vectorizer.build_analyzer()

word_embeddings = Word2Vec([analyzer(sentence) for sentence in clean_sentences], min_count=1)

sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.wv[w] for w in analyzer(i)])/(len(i.split())+0.001)
  else:
    v = np.zeros((1,))
  sentence_vectors.append(v)
#sentence_vectors

Word2Vec(vocab=238, vector_size=100, alpha=0.025)


Compute the similarity between the sentences and store the values in asquare similarity matrix

In [23]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

Apply the PageRank algorithm as usual but using the similarity matrix

In [24]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank_numpy(nx_graph)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  


In [30]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
print('TextRank Score - Comment')
for i in range(len(sentences)):
  print(str(ranked_sentences[i][0]) + ' - ' + ranked_sentences[i][1])

TextRank Score - Comment
11284492996178.58 - Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.
11059673196972.896 - A entrega foi no prazo, as americanas estão de parabéns.
10448089544935.06 - Nunca tive problema com a site americanas mais essa compra foi com o parceiro Miracurl que não cumpriu com seu deveres.
10429566458731.426 - A entrega foi no prazo estipulado, nem antes e nem depois.
9765629621361.48 - O produto não gela.
8123219411057.602 - Só quero que entregue o produto ou então que reembolso o dinheiro.Pois a compra foi paga via cartão.
6387374258807.039 - Nunca coloquei nada natural para ver se gela.
6065229919203.168 - Comprei a prancha babyliss Pro Nano dia 27/02 é até o momento não recebi o produto o prazo de entrega foi até 16/03.
3419111037379.5874 - SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANELAS ELÉTRICAS.
2720434130058.16 - NESSA PANELA  É MUITO MAIS RÁPIDO, 